In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
df - pd.read_csv(test_data)
kaggle_data = pd.read_csv(train_data)

In [ ]:
#Identifying columns in test data not in train
[col for col in df.columns if col not in kaggle_data.columns]

In [ ]:
#Another way to check
set(df.columns) - set(kaggle_data.columns)

### Baseline model

In [ ]:
df['SalePrice'].mean()

In [ ]:
#Median more robust to outliers
df['SalePrice'].median()

### Create SalePrice column in test data

In [ ]:
kaggle_data['SalePrice'] = df['SalePrice'].mean()

In [ ]:
#check training data
kaggle_data.head(2)


### Submission format is only the ID & SalePrice

In [ ]:
#Assign to variable submissions
submission = kaggle_data[['Id', 'SalePrice']]
submission.head(2)
    #When I do this, it will create an index column

In [ ]:
#Setting the index so it's ID
submission.set_index('Id', inplace = True)
submission.head(2)

In [ ]:
submission.to_csv()

### MLR model

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
df.isna().sum().sort_values(ascending = False)
    #See there's a lot of null values

In [ ]:
df.dropna(axis = 1)
df.dropna(axis = 1).shape
    #Change axis to 1 to drop columns

In [ ]:
df._get_numeric_data()
df._get_numeric_data().shape
    #Will just get numeric data

In [ ]:
df = df.dropna(axis = 1)._get_numeric_data()
df.shape

In [ ]:
#Run correlation matrix
df.corr()[['SalePrice']].sort_values('SalePrice', ascending = False)

In [ ]:
# Rename columns to be pythonic (no spaces, underscores, lower case), using list comprehension
df.columns = [col.replace(' ', '_').lower() for col in df.columns]
df.head

### Create our model

In [ ]:
features = ['fireplaces', 'first_flr_sf', 'gr_liv_area']

X = df[features]
y = df['saleprice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 22)

In [ ]:
lr = LinearRegression()

lr.fit(X_train, y_train)
print(cross_val_score(lr, X_train, y_train).mean() )
lr.score(X_train, y_train), lr.score(X_test, y_test)

In [ ]:
### Scores coming out around .57
#Has bias, but not overfit - since they're close together

### Now we need to generate predictions on the test data

In [ ]:
#Will create error since renamed cols in train data, but not test data
lr.predict(kaggle_data[[features])

In [ ]:
#Rename kaggle columns
kaggle_data.columns = [col.replace(' ', '_').lower() for col in kaggle_data.columns]

In [ ]:
predictions = lr.predict(kaggle_data[features])

In [ ]:
predictions[:5]

In [ ]:
#Replacing sale price data with predictions from model
kaggle_data['saleprice'] = predictions

In [ ]:
#index = False will not add an index
kaggel_data[['id', 'saleprice']].to_csv('3features.csv', index = False)